In [1]:
import sqlalchemy as db
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

from mclLib.featurenames import *
from mclLib.server_info import *
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [2]:
engine = create_engine(mysql_server_uri)
conn = engine.connect()

''' get db data '''
br_full_data = pd.read_sql_table('buyreports', conn)

''' db data filter '''
get_filter = ( br_full_data['isAllBuyed'] == 1) & ( br_full_data['isAllSelled'] == 1) 
split_filter = (br_full_data['dTradeTime'] < '2023-03-01')
br = br_full_data.loc[get_filter]
br_train = br.loc[split_filter]
br_test =  br.loc[~split_filter]


''' get features name'''
feature_names_102 =  f_name_102

''' set X data '''
train_x = br_train[feature_names_102].to_numpy(dtype=np.float64)
test_x = br_test[feature_names_102].to_numpy(dtype=np.float64)
''' set y data '''
y_condition = (br_train['fMaxPowerAfterBuyWhile10'] < 0.035)
train_y = np.where(y_condition, 1, 0)

y_condition = (br_test['fMaxPowerAfterBuyWhile10'] < 0.035)
test_y = np.where(y_condition, 1, 0)

In [3]:
import numpy as np
import sklearn.metrics
from sklearn.model_selection import train_test_split

from ray import tune
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.lightgbm import TuneReportCheckpointCallback

import gc

def trainable(config, train_set, test_set):
    rf = RandomForestClassifier(
                            n_estimators=config["n_estimators"], 
                            criterion=config["criterion"], 
                            max_depth=config["max_depth"],
                            max_features=config["max_features"],
                            max_samples=config["max_samples"]
                            )
    rf.fit(train_set[0] , train_set[1])
    y_pred = rf.predict(test_set[0])
    return { "score" : roc_auc_score(test_set[1], y_pred)}

   
trainable = tune.with_parameters(trainable, train_set=(train_x, train_y), test_set=(test_x, test_y))
                                    
config = {
    "n_estimators": tune.randint(40, 200),
    "criterion": tune.choice(["gini", "entropy"]),
    "max_depth": tune.randint(3, 7),
    "max_features": tune.uniform(0.4, 1.0),
    "max_samples": tune.uniform(0.4, 1.0),
}

tuner = tune.Tuner(
    trainable,
    tune_config=tune.TuneConfig(
        metric="score",
        mode="max",
        scheduler=ASHAScheduler(),
        num_samples=50,
    ),
    param_space=config,
    
)
results = tuner.fit()


2023-03-20 20:34:48,606	INFO worker.py:1553 -- Started a local Ray instance.


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,score,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
trainable_3ab1e_00003,2023-03-20_20-38-01,True,,fe95173cfccb48d4bb3653a5c4f30a55,"3_criterion=entropy,max_depth=5,max_features=0.4841,max_samples=0.9954,n_estimators=44",DESKTOP-U3J9K27,1,127.0.0.1,17980,0.508301,150.887,150.887,150.887,1679312281,0,,1,3ab1e_00003,0.00999832


2023-03-20 20:50:23,419	WARNING tune.py:146 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2023-03-20 20:50:24,678	ERROR tune.py:794 -- Trials did not complete: [trainable_3ab1e_00006, trainable_3ab1e_00007, trainable_3ab1e_00008, trainable_3ab1e_00009, trainable_3ab1e_00010, trainable_3ab1e_00011, trainable_3ab1e_00012, trainable_3ab1e_00013, trainable_3ab1e_00014, trainable_3ab1e_00015, trainable_3ab1e_00016, trainable_3ab1e_00017, trainable_3ab1e_00018, trainable_3ab1e_00019, trainable_3ab1e_00020, trainable_3ab1e_00021, trainable_3ab1e_00022, trainable_3ab1e_00023, trainable_3ab1e_00024, trainable_3ab1e_00025]
2023-03-20 20:50:24,678	INFO tune.py:798 -- Total run time: 932.54 seconds (932.24 seconds for the tuning loop).
2023-03-20 20:50:24,679	WARNING tune.py:804 -- Experiment has been interrupted, but the most recent state was saved. You 

In [7]:

print("Best hyperparameters found were: ", results.get_best_result().config)

Best hyperparameters found were:  {'n_estimators': 86, 'criterion': 'entropy', 'max_depth': 6, 'max_features': 0.7244048002153054, 'max_samples': 0.9851169783502004}


In [8]:
results.get_dataframe()

,f1_score,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,trial_id,experiment_id,date,timestamp,...,time_since_restore,timesteps_since_restore,iterations_since_restore,warmup_time,config/criterion,config/max_depth,config/max_features,config/max_samples,config/n_estimators,logdir
0,0.923610,280.449991,False,NaN,NaN,1,554cc_00000,24a567e772df448f990ff87c1ac7e6e9,2023-03-20_19-07-25,1679306845,...,280.449991,0,1,0.011004,gini,6,0.829796,0.597244,67,C:\Users\all\ray_results\trainable_2023-03-20_...
1,0.923553,95.552997,False,NaN,NaN,1,554cc_00001,c3bf038a6bf941e9b21af3eabb8b0112,2023-03-20_19-04-34,1679306674,...,95.552997,0,1,0.011003,entropy,3,0.608185,0.440454,60,C:\Users\all\ray_results\trainable_2023-03-20_...
2,0.923553,240.687999,False,NaN,NaN,1,554cc_00002,43a41e3cea49409b9cba487cda9aca66,2023-03-20_19-07-09,1679306829,...,240.687999,0,1,0.010001,gini,3,0.968063,0.413462,126,C:\Users\all\ray_results\trainable_2023-03-20_...
3,0.923553,276.616996,True,NaN,NaN,1,554cc_00003,482e72efb51849cd8fdb9e578b412aa2,2023-03-20_19-07-57,1679306877,...,276.616996,0,1,0.017002,entropy,5,0.728386,0.665766,69,C:\Users\all\ray_results\trainable_2023-03-20_...
4,0.923464,132.037000,True,NaN,NaN,1,554cc_00004,c3bf038a6bf941e9b21af3eabb8b0112,2023-03-20_19-06-46,1679306806,...,132.037000,0,1,0.011003,gini,5,0.616660,0.881279,41,C:\Users\all\ray_results\trainable_2023-03-20_...
5,0.923404,665.242067,True,NaN,NaN,1,554cc_00005,c3bf038a6bf941e9b21af3eabb8b0112,2023-03-20_19-17-51,1679307471,...,665.242067,0,1,0.011003,entropy,6,0.757555,0.461964,166,C:\Users\all\ray_results\trainable_2023-03-20_...
6,0.923618,228.815000,False,NaN,NaN,1,554cc_00006,43a41e3cea49409b9cba487cda9aca66,2023-03-20_19-10-58,1679307058,...,228.815000,0,1,0.010001,gini,4,0.908134,0.671581,70,C:\Users\all\ray_results\trainable_2023-03-20_...
7,0.923407,707.694066,True,NaN,NaN,1,554cc_00007,24a567e772df448f990ff87c1ac7e6e9,2023-03-20_19-19-13,1679307553,...,707.694066,0,1,0.011004,entropy,6,0.738686,0.491783,174,C:\Users\all\ray_results\trainable_2023-03-20_...
8,0.923486,322.542065,True,NaN,NaN,1,554cc_00008,482e72efb51849cd8fdb9e578b412aa2,2023-03-20_19-13-19,1679307199,...,322.542065,0,1,0.017002,gini,5,0.909570,0.849358,67,C:\Users\all\ray_results\trainable_2023-03-20_...
9,0.923597,842.224067,False,NaN,NaN,1,554cc_00009,43a41e3cea49409b9cba487cda9aca66,2023-03-20_19-25-01,1679307901,...,842.224067,0,1,0.010001,entropy,6,0.928336,0.789714,124,C:\Users\all\ray_results\trainable_2023-03-20_...


In [10]:
results.get_best_result().predict()



AttributeError: 'Result' object has no attribute 'predict'